# Preprocess and clean train services data

In [1]:
# imports
import csv
import numpy as np
import pandas as pd
from datetime import datetime, date

In [2]:
# load dataframe from input file
df = pd.read_csv(r"C:\Users\pablo\ProjectsData\HSTrainWebScraping\20220809\full_file.txt", sep='|')

In [3]:
# remove duplicate entries
df.drop_duplicates()
df.head()

,origin_station,destination_station,travel_date,departure_time,arrival_time,price,company,search_date,search_time
0,BARCELONA,MADRID,11/09/2022,06:45,09:15,25€,OUIGO,10/07/2022,03:08
1,BARCELONA,MADRID,11/09/2022,10:40,13:25,25€,OUIGO,10/07/2022,03:08
2,BARCELONA,MADRID,11/09/2022,13:38,16:30,39€,OUIGO,10/07/2022,03:08
3,BARCELONA,MADRID,11/09/2022,17:45,20:15,45€,OUIGO,10/07/2022,03:08
4,BARCELONA,MADRID,11/09/2022,20:38,23:30,39€,OUIGO,10/07/2022,03:08


### Generate full train ID
Composed of origin, destination, travel date, departure time and company

In [4]:
df['train_id'] = df['origin_station'] + ',' + \
    df['destination_station'] + ',' + \
    df['travel_date'] + ',' + \
    df['departure_time'] + ',' + \
    df['company']

df.head()

,origin_station,destination_station,travel_date,departure_time,arrival_time,price,company,search_date,search_time,train_id
0,BARCELONA,MADRID,11/09/2022,06:45,09:15,25€,OUIGO,10/07/2022,03:08,"BARCELONA,MADRID,11/09/2022,06:45,OUIGO"
1,BARCELONA,MADRID,11/09/2022,10:40,13:25,25€,OUIGO,10/07/2022,03:08,"BARCELONA,MADRID,11/09/2022,10:40,OUIGO"
2,BARCELONA,MADRID,11/09/2022,13:38,16:30,39€,OUIGO,10/07/2022,03:08,"BARCELONA,MADRID,11/09/2022,13:38,OUIGO"
3,BARCELONA,MADRID,11/09/2022,17:45,20:15,45€,OUIGO,10/07/2022,03:08,"BARCELONA,MADRID,11/09/2022,17:45,OUIGO"
4,BARCELONA,MADRID,11/09/2022,20:38,23:30,39€,OUIGO,10/07/2022,03:08,"BARCELONA,MADRID,11/09/2022,20:38,OUIGO"


### Replace full train ID by standard ID
Create equivalence between full train ID and standard ID, which is a cleaner ID: "T####"

Replace full train ID by standard ID in dataframe.

In [5]:
set_train_full_ids = set(list(df['train_id']))

id_digits = len(str(len(set_train_full_ids)))

train_full_id_to_new = {train_full_id: 'T{}'.format(str(i).zfill(id_digits)) 
                        for i, train_full_id in enumerate(set_train_full_ids)}

for i in range(len(df['train_id'])):
    df['train_id'][i] = train_full_id_to_new[df['train_id'][i]]

df.head()

,origin_station,destination_station,travel_date,departure_time,arrival_time,price,company,search_date,search_time,train_id
0,BARCELONA,MADRID,11/09/2022,06:45,09:15,25€,OUIGO,10/07/2022,03:08,T1493
1,BARCELONA,MADRID,11/09/2022,10:40,13:25,25€,OUIGO,10/07/2022,03:08,T2361
2,BARCELONA,MADRID,11/09/2022,13:38,16:30,39€,OUIGO,10/07/2022,03:08,T1348
3,BARCELONA,MADRID,11/09/2022,17:45,20:15,45€,OUIGO,10/07/2022,03:08,T0500
4,BARCELONA,MADRID,11/09/2022,20:38,23:30,39€,OUIGO,10/07/2022,03:08,T2532


### Days difference between search and train
Calculate number of days between search date and travel date and add as a new column.

In [6]:
def days_difference(row):
    dt1 = datetime.strptime(row['travel_date'], '%d/%m/%Y').date()
    dt2 = datetime.strptime(row['search_date'], '%d/%m/%Y').date()
    return (dt1 - dt2).days

df['days_diff'] = df.apply(lambda row: days_difference(row), axis=1)
df.head(10)

,origin_station,destination_station,travel_date,departure_time,arrival_time,price,company,search_date,search_time,train_id,days_diff
0,BARCELONA,MADRID,11/09/2022,06:45,09:15,25€,OUIGO,10/07/2022,03:08,T1493,63
1,BARCELONA,MADRID,11/09/2022,10:40,13:25,25€,OUIGO,10/07/2022,03:08,T2361,63
2,BARCELONA,MADRID,11/09/2022,13:38,16:30,39€,OUIGO,10/07/2022,03:08,T1348,63
3,BARCELONA,MADRID,11/09/2022,17:45,20:15,45€,OUIGO,10/07/2022,03:08,T0500,63
4,BARCELONA,MADRID,11/09/2022,20:38,23:30,39€,OUIGO,10/07/2022,03:08,T2532,63
5,BARCELONA,MADRID,16/07/2022,06:45,09:15,59€,OUIGO,10/07/2022,03:08,T0152,6
6,BARCELONA,MADRID,16/07/2022,10:40,13:25,85€,OUIGO,10/07/2022,03:08,T2439,6
7,BARCELONA,MADRID,16/07/2022,13:45,16:30,49€,OUIGO,10/07/2022,03:08,T0379,6
8,BARCELONA,MADRID,16/07/2022,17:45,20:15,49€,OUIGO,10/07/2022,03:08,T1573,6
9,BARCELONA,MADRID,16/07/2022,20:38,23:30,39€,OUIGO,10/07/2022,03:08,T0944,6


### Calculate weekday

In [7]:
def weekday(search_date):
    days_dict = {
        0: "Monday",
        1: "Tuesday",
        2: "Wednesday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
        6: "Sunday"
    }
    return days_dict[datetime.strptime(search_date, '%d/%m/%Y').date().weekday()]

df['weekday'] = df.apply(lambda row: weekday(row['travel_date']), axis=1)
df.head(10)

,origin_station,destination_station,travel_date,departure_time,arrival_time,price,company,search_date,search_time,train_id,days_diff,weekday
0,BARCELONA,MADRID,11/09/2022,06:45,09:15,25€,OUIGO,10/07/2022,03:08,T1493,63,Sunday
1,BARCELONA,MADRID,11/09/2022,10:40,13:25,25€,OUIGO,10/07/2022,03:08,T2361,63,Sunday
2,BARCELONA,MADRID,11/09/2022,13:38,16:30,39€,OUIGO,10/07/2022,03:08,T1348,63,Sunday
3,BARCELONA,MADRID,11/09/2022,17:45,20:15,45€,OUIGO,10/07/2022,03:08,T0500,63,Sunday
4,BARCELONA,MADRID,11/09/2022,20:38,23:30,39€,OUIGO,10/07/2022,03:08,T2532,63,Sunday
5,BARCELONA,MADRID,16/07/2022,06:45,09:15,59€,OUIGO,10/07/2022,03:08,T0152,6,Saturday
6,BARCELONA,MADRID,16/07/2022,10:40,13:25,85€,OUIGO,10/07/2022,03:08,T2439,6,Saturday
7,BARCELONA,MADRID,16/07/2022,13:45,16:30,49€,OUIGO,10/07/2022,03:08,T0379,6,Saturday
8,BARCELONA,MADRID,16/07/2022,17:45,20:15,49€,OUIGO,10/07/2022,03:08,T1573,6,Saturday
9,BARCELONA,MADRID,16/07/2022,20:38,23:30,39€,OUIGO,10/07/2022,03:08,T0944,6,Saturday


### Boolean field for full trains

In [8]:
def full_train(row):
    full_train = 'completo' in row['price'].lower()
    return full_train

df['full_train'] = df.apply(lambda row: full_train(row), axis=1)
df.head(10)

,origin_station,destination_station,travel_date,departure_time,arrival_time,price,company,search_date,search_time,train_id,days_diff,weekday,full_train
0,BARCELONA,MADRID,11/09/2022,06:45,09:15,25€,OUIGO,10/07/2022,03:08,T1493,63,Sunday,False
1,BARCELONA,MADRID,11/09/2022,10:40,13:25,25€,OUIGO,10/07/2022,03:08,T2361,63,Sunday,False
2,BARCELONA,MADRID,11/09/2022,13:38,16:30,39€,OUIGO,10/07/2022,03:08,T1348,63,Sunday,False
3,BARCELONA,MADRID,11/09/2022,17:45,20:15,45€,OUIGO,10/07/2022,03:08,T0500,63,Sunday,False
4,BARCELONA,MADRID,11/09/2022,20:38,23:30,39€,OUIGO,10/07/2022,03:08,T2532,63,Sunday,False
5,BARCELONA,MADRID,16/07/2022,06:45,09:15,59€,OUIGO,10/07/2022,03:08,T0152,6,Saturday,False
6,BARCELONA,MADRID,16/07/2022,10:40,13:25,85€,OUIGO,10/07/2022,03:08,T2439,6,Saturday,False
7,BARCELONA,MADRID,16/07/2022,13:45,16:30,49€,OUIGO,10/07/2022,03:08,T0379,6,Saturday,False
8,BARCELONA,MADRID,16/07/2022,17:45,20:15,49€,OUIGO,10/07/2022,03:08,T1573,6,Saturday,False
9,BARCELONA,MADRID,16/07/2022,20:38,23:30,39€,OUIGO,10/07/2022,03:08,T0944,6,Saturday,False


### New field with interval of days to trip

In [9]:
# days diff interval
def days_interval(num):
    if num <= 1:
        return '0-1 day'
    elif num <= 3:
        return '2-3 days'
    elif num <= 7:
        return '4-7 days'
    elif num <= 14:
        return '1-2 weeks'
    elif num <= 21:
        return '2-3 weeks'
    elif num <= 28:
        return '3-4 weeks'
    elif num <= 45:
        return '1-1.5 months'
    elif num <= 61:
        return '1.5-2 months'
    elif num <= 91:
        return '2-3 months'
    else:
        return '>3 months'

df['days_diff_interval'] = df.apply(lambda row: days_interval(row['days_diff']), axis=1)

### New field with price group

In [10]:
# price group
def price_group(num):
    try:
        num = num.replace('€', '')
        num = num.replace(',00 ', '')
        num = int(num)
        if num < 20:
            return '<20€'
        elif num <= 35:
            return '20-35€'
        elif num <= 50:
            return '36-50€'
        elif num <= 80:
            return '51-80€'
        else:
            return '>80€'
    except:
        return np.NaN

df['price_interval'] = \
    df.apply(lambda row: price_group(row['price']), axis=1)

df.head()

,origin_station,destination_station,travel_date,departure_time,arrival_time,price,company,search_date,search_time,train_id,days_diff,weekday,full_train,days_diff_interval,price_interval
0,BARCELONA,MADRID,11/09/2022,06:45,09:15,25€,OUIGO,10/07/2022,03:08,T1493,63,Sunday,False,2-3 months,20-35€
1,BARCELONA,MADRID,11/09/2022,10:40,13:25,25€,OUIGO,10/07/2022,03:08,T2361,63,Sunday,False,2-3 months,20-35€
2,BARCELONA,MADRID,11/09/2022,13:38,16:30,39€,OUIGO,10/07/2022,03:08,T1348,63,Sunday,False,2-3 months,36-50€
3,BARCELONA,MADRID,11/09/2022,17:45,20:15,45€,OUIGO,10/07/2022,03:08,T0500,63,Sunday,False,2-3 months,36-50€
4,BARCELONA,MADRID,11/09/2022,20:38,23:30,39€,OUIGO,10/07/2022,03:08,T2532,63,Sunday,False,2-3 months,36-50€


### Write to output file

In [11]:
output_path = r"C:\Users\pablo\ProjectsData\HSTrainWebScraping\20220809\full_file_cleaned.txt"
with open(output_path,'w') as output_file:
    output_file.write(df.to_csv(sep='|', index=False, line_terminator='\n'))